# TRANSACTION DATA SIMULATOR

In [1]:
!pip install randomtimestamp

In [2]:
import pandas as pd
import numpy as np

import os
import datetime

import random
from decimal import *
from scipy.stats import poisson

from datetime import datetime
from randomtimestamp import random_time 

# Customer profiles generation

In [3]:
def generate_customer_profiles_table(n_customers, random_state):
    
    x = []
    y = []
    
    mean_amount = []
    std_amount = []
    mean_nb_tx_per_day = []
    
    # 0 - creating dataframe
    df = pd.DataFrame(columns=['CUSTOMER_ID', 'x_customer_id', 'y_customer_id', 'mean_amount', 'std_amount', 'mean_nb_tx_per_day'])
    
    # 1 - terminal ID
    terminal_id = [*range(0, n_customers)]
    
    df['CUSTOMER_ID'] = terminal_id
    
    
    # 2 - x coordinate
    random.seed(random_state)
    #random.randint(0, 100)
    
    for i in range(0,n_customers):
        n = random.uniform(0,100)
        x.append(n)
    
    
    df['x_customer_id'] = x
    
    # 3 - y coordinate
    
    for j in range(0,n_customers):
        n = random.uniform(0,100)
        y.append(n)
    
    
    df['y_customer_id'] = y
    
    # 4 mean_amount
    for z in range(0,n_customers):
        n = random.uniform(5,100)
        mean_amount.append(n)
        
    df['mean_amount'] = mean_amount
    
    # 5 std
    for t in mean_amount:
        n = t / 2
        std_amount.append(n)
    df['std_amount'] = std_amount
    
    # 6 mean_nb_tx_per_day
    
    for r in range(0,n_customers):
        n = random.uniform(0,4)
        mean_nb_tx_per_day.append(n)
    df['mean_nb_tx_per_day'] = mean_nb_tx_per_day
    
    
    
    return df

customer_profiles_table = generate_customer_profiles_table(5, 42)
print(customer_profiles_table)



   CUSTOMER_ID  x_customer_id  y_customer_id  mean_amount  std_amount  \
0            0      63.942680      67.669949    25.770608   12.885304   
1            1       2.501076      89.217957    53.008752   26.504376   
2            2      27.502932       8.693883     7.520917    3.760459   
3            3      22.321074      42.192182    23.889577   11.944788   
4            4      73.647121       2.979722    66.739022   33.369511   

   mean_nb_tx_per_day  
0            2.179766  
1            0.881762  
2            2.357063  
3            3.237722  
4            0.025995  


# Terminal profiles generation

In [15]:
def generate_terminal_profiles_table(n_terminals, random_state):
    x = []
    y = []
    
    # 0 - creating dataframe
    df = pd.DataFrame(columns=['TERMINAL_ID', 'x_terminal_id', 'y_terminal_id'])
    
    # 1 - terminal ID
    terminal_id = [*range(0, n_terminals)]
    
    df['TERMINAL_ID'] = terminal_id
    
    
    # 2 - x coordinate
    random.seed(random_state)
    #random.randint(0, 100)
    
    for i in range(0,n_terminals):
        n = random.uniform(0,100)
        x.append(n)
    
    
    df['x_terminal_id'] = x
    
    # 3 - y coordinate
    
    for j in range(0,n_terminals):
        n = random.uniform(0,100)
        y.append(n)
    
    
    df['y_terminal_id'] = y
    
  
    return df

terminal_profiles_table = generate_terminal_profiles_table(5, 0)
print(terminal_profiles_table)

   TERMINAL_ID  x_terminal_id  y_terminal_id
0            0      84.442185      40.493414
1            1      75.795440      78.379859
2            2      42.057158      30.331273
3            3      25.891675      47.659695
4            4      51.127472      58.338204


# Association of customer profiles to terminals

In [16]:
x_y_terminals = terminal_profiles_table[['x_terminal_id','y_terminal_id']].values.astype(float)
#https://www.datacamp.com/tutorial/pandas-apply


In [17]:
def get_list_terminals_within_radius(row, x_y_terminals, r):
    available_terminals = []
    # customer coordinates + radius value must be >= than x or y coordinate of terminal
    # if yes, terminal is added to the list
    x_coordinate_customer_plus_radius = row['x_customer_id'] + r
    y_coordinate_customer_plus_radius = row['y_customer_id'] + r
    
    for j in range(1, len(x_y_terminals)):
        if x_y_terminals[j][0] <= x_coordinate_customer_plus_radius and x_y_terminals[j][1] <= y_coordinate_customer_plus_radius:
            available_terminals.append(j)
   
    return available_terminals
    
get_list_terminals_within_radius(customer_profiles_table.iloc[4], x_y_terminals=x_y_terminals, r=50)
#list_of_possible_terminals = get_list_terminals_within_radius(3, x_y_terminals=x_y_terminals, r=50)
#list_of_possible_terminals

[1, 2, 3, 4]

In [18]:
customer_profiles_table['available_terminals'] = customer_profiles_table.apply(lambda x: get_list_terminals_within_radius(x, x_y_terminals = x_y_terminals, r=50), axis=1)
customer_profiles_table

,CUSTOMER_ID,x_customer_id,y_customer_id,mean_amount,std_amount,mean_nb_tx_per_day,available_terminals
0,0,63.942680,12.971314,95.059772,47.529886,2.246639,"[2, 3, 4]"
1,1,2.501076,56.263434,90.910767,45.455384,3.978768,"[2, 3, 4]"
2,2,27.502932,51.970580,71.793629,35.896814,0.912441,"[1, 2, 3, 4]"
3,3,22.321074,63.185761,77.729551,38.864776,2.426924,"[2, 3, 4]"
4,4,73.647121,49.250445,8.359397,4.179699,1.303112,"[1, 2, 3, 4]"
...,...,...,...,...,...,...,...
4995,4995,60.884461,92.417426,44.681848,22.340924,2.624950,"[1, 2, 3, 4]"
4996,4996,31.648856,66.333613,89.702712,44.851356,1.286676,"[1, 2, 3, 4]"
4997,4997,52.539123,87.827018,65.983916,32.991958,1.800927,"[1, 2, 3, 4]"
4998,4998,99.502434,23.561720,21.521889,10.760945,2.946982,"[2, 3, 4]"


# Generation of transactions

In [19]:
def generate_transactions_table(profile, start_date, nb_days):
    #profile 
    #start_date 
    #nb_days
    
    #TX_DATETIME, TX_AMOUNT, TX_TIME_SECONDS, TX_TIME_DAYS
    
    number_of_transactions_each_day = 0
    sum_days = 0
    TX_DATETIME = []
    CUSTOMER_ID = []
    
    TERMINAL_ID = []
    TX_AMOUNT = []
    
    TX_TIME_SECONDS = []
    TX_TIME_DAYS = []
    
    # 0 - creating dataframe
    df = pd.DataFrame(columns=['TX_DATETIME', 'CUSTOMER_ID', 'TERMINAL_ID', 'TX_AMOUNT', 'TX_TIME_SECONDS', 'TX_TIME_DAYS'])
    
    
    dates = []
    date_time_list = []
    dates_by_occurence = []
    seconds = []
    ind = []


    number_of_transactions_each_day = 0
    sum_days = 0
    k = 0
    l = 0
    p = 0
    pp = 0

    # Generating different dates
    dt_list = pd.date_range(start='2018-04-01', periods=nb_days, freq='D')
    # Translating format to str
    for i in dt_list:
        year = i.strftime("%Y")
        month = i.strftime("%m")
        day = i.strftime("%d")
        date = year + '-' + month + '-' + day
        dates.append(date)

    # Fixing random values for time
    random.seed(42)

    # Fixing random transaction occurence by day
    np.random.seed(42)
    number_of_transactions_each_day = np.random.poisson(2.179533,nb_days)

    # Calculating final number of transaction occurences        
    for j in number_of_transactions_each_day:
        sum_days += j

    # Association of transaction of occurence each day and the dates, e.g. [day_x] * 4
    while k < len(number_of_transactions_each_day) and l < len(dates):
        new_val = number_of_transactions_each_day[k] * [dates[l]]
        dates_by_occurence.append(new_val)
        k += 1
        l += 1

    # Unnesting the list
    flatten_lambda = lambda lst: (item for sublist in lst for item in sublist)
    dates_fin = list(flatten_lambda(dates_by_occurence))
    
    
    # TX_TIME_DAYS Generation
    
    for i in range(0,len(number_of_transactions_each_day)):
        ind.append(str(i))

    
    while p < len(number_of_transactions_each_day) and pp < len(ind):
        new_val = number_of_transactions_each_day[p] * [ind[pp]]
        TX_TIME_DAYS.append(new_val)
        p += 1
        pp += 1


    flatten_lambda = lambda lst: (item for sublist in lst for item in sublist)
    TX_TIME_DAYS_fin = list(flatten_lambda(TX_TIME_DAYS))
    

    # Association of tx occurence days and randomly generated time
    for z in dates_fin:
        a = random_time()
        time = a.strftime("%H:%M:%S")
        time_z = int(time[0:2]) * 60 * 60 + int(time[3:5]) * 60 + int(time[6:8])
        seconds.append(time_z)
        date_time_list.append(z + ' ' + time)
        
    
    
    for q in range(len(TX_TIME_DAYS_fin)):
        
        random_num = random.choice(profile['available_terminals'])
        TERMINAL_ID.append(random_num)
    
    mu = abs(profile['mean_amount'])
    sigma = abs(profile['std_amount'])
     
    #mu, sigma = 0, 0.1 # mean and standard deviation
    s = np.random.normal(mu, sigma, len(TX_TIME_DAYS_fin))
    #print(s)
    #print(abs(mu - np.mean(s)))
    #0.0  # may vary
    
    df['TX_DATETIME'] = date_time_list
    df['TX_TIME_SECONDS'] = seconds
    df['TX_TIME_DAYS'] = TX_TIME_DAYS_fin
    df['CUSTOMER_ID'] = int(profile['CUSTOMER_ID'])
    df['TERMINAL_ID'] = TERMINAL_ID
    df['TX_AMOUNT'] = abs(s)


    return df
    
transaction_table_customer_0 = generate_transactions_table(customer_profiles_table.iloc[0], start_date = "2018-04-01", nb_days = 5)
transaction_table_customer_0

,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS
0,2018-04-01 23:16:50,0,2,29.114385,83810,0
1,2018-04-01 04:03:12,0,2,140.230734,14592,0
2,2018-04-01 00:54:38,0,2,133.865313,3278,0
3,2018-04-01 10:00:48,0,2,22.630119,36048,0
4,2018-04-02 08:54:58,0,2,74.714791,32098,1
5,2018-04-03 08:07:36,0,4,59.773259,29256,2
6,2018-04-03 05:04:49,0,4,61.629921,18289,2
7,2018-04-03 03:43:54,0,2,6.636154,13434,2
8,2018-04-04 19:51:22,0,4,65.140899,71482,3
9,2018-04-04 03:09:55,0,2,123.469357,11395,3


In [29]:
transactions_df = customer_profiles_table.groupby('CUSTOMER_ID').apply(lambda x : generate_transactions_table(x.iloc[0], start_date = "2018-04-01", nb_days=5)).reset_index(drop=True)
transactions_df

,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS
0,2018-04-01 23:16:50,0,1,7.892880,83810,0
1,2018-04-01 04:03:12,0,1,38.016410,14592,0
2,2018-04-01 00:54:38,0,1,36.290751,3278,0
3,2018-04-01 10:00:48,0,2,6.135002,36048,0
4,2018-04-02 08:54:58,0,2,20.255104,32098,1
5,2018-04-03 08:07:36,0,1,16.204470,29256,2
6,2018-04-03 05:04:49,0,2,16.707809,18289,2
7,2018-04-03 03:43:54,0,4,1.799055,13434,2
8,2018-04-04 19:51:22,0,2,17.659631,71482,3
9,2018-04-04 03:09:55,0,4,33.472417,11395,3


In [9]:
!pip install pandarallel

In [10]:
from pandarallel import pandarallel
# Initialization
pandarallel.initialize()

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


# Larger scale

In [11]:
def generate_dataset(n_customers, n_terminals, nb_days, start_date, r):
    
    #df.parallel_apply(func)
    
    # 1 customers 1
    customer_profiles_table = generate_customer_profiles_table(n_customers, 42)
    # 2 terminals 1
    terminal_profiles_table = generate_terminal_profiles_table(n_terminals, 42)
    # 2.1 
    x_y_terminals = terminal_profiles_table[['x_terminal_id','y_terminal_id']].values.astype(float)
    # 3 association 
    customer_profiles_table['available_terminals']=customer_profiles_table.parallel_apply(lambda x : get_list_terminals_within_radius(x, x_y_terminals=x_y_terminals, r=r), axis=1)
    #customer_profiles_table
    # 4 transactions ??
    transaction_table_customer_0=generate_transactions_table(customer_profiles_table.iloc[0], start_date, nb_days)
    # 5 association of transactions to customers - here NEED to add transactions ID 1
    transactions_df=customer_profiles_table.groupby('CUSTOMER_ID').parallel_apply(lambda x : generate_transactions_table(x.iloc[0], start_date = start_date, nb_days=nb_days)).reset_index(drop=True)
    # 6 transactions_df - adding transaction ID column
    transactions_df['TRANSACTION_ID'] = [*range(0, len(transactions_df.index))]
    # 6.1 moving transaction ID column to the first position 
    transactions_df.insert(0, 'TRANSACTION_ID', transactions_df.pop('TRANSACTION_ID'))
  
    
    return customer_profiles_table, terminal_profiles_table, transactions_df

(customer_profiles_table, terminal_profiles_table, transactions_df)=\
    generate_dataset(n_customers = 5000, n_terminals = 10000, nb_days=183,start_date="2018-04-01", r=5)



In [12]:
transactions_df.shape

(2010000, 7)

In [13]:
transactions_df

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS
0,0,2018-04-01 23:16:50,0,1996,89.217811,83810,0
1,1,2018-04-01 04:03:12,0,1370,91.480598,14592,0
2,2,2018-04-01 00:54:38,0,2694,42.717394,3278,0
3,3,2018-04-01 10:00:48,0,9071,118.238827,36048,0
4,4,2018-04-02 08:54:58,0,669,130.582089,32098,1
...,...,...,...,...,...,...,...
2009995,2009995,2018-09-29 16:05:12,4999,9463,38.885767,57912,181
2009996,2009996,2018-09-29 20:05:09,4999,4274,29.894204,72309,181
2009997,2009997,2018-09-30 15:34:17,4999,6320,20.708005,56057,182
2009998,2009998,2018-09-30 20:25:19,4999,6116,16.515782,73519,182


In [52]:
transactions_df.dtypes

TRANSACTION_ID         int64
TX_DATETIME           object
CUSTOMER_ID            int64
TERMINAL_ID            int64
TX_AMOUNT            float64
TX_TIME_SECONDS        int64
TX_TIME_DAYS          object
TX_FRAUD               int64
TX_FRAUD_SCENARIO      int64
dtype: object

In [57]:
def add_frauds(customer_profiles_table, terminal_profiles_table, transactions_df):
    
    # By default, all transactions are genuine
    transactions_df['TX_FRAUD']=0
    transactions_df['TX_FRAUD_SCENARIO']=0
    
    # Scenario 1
    transactions_df.loc[transactions_df.TX_AMOUNT>220, 'TX_FRAUD']=1
    transactions_df.loc[transactions_df.TX_AMOUNT>220, 'TX_FRAUD_SCENARIO']=1
    nb_frauds_scenario_1=transactions_df.TX_FRAUD.sum()
    print("Number of frauds from scenario 1: "+str(nb_frauds_scenario_1))
    
    transactions_df['TX_TIME_DAYS'] = transactions_df['TX_TIME_DAYS'].astype(str).astype(int)
    
    #print(transactions_df.dtypes)
    # Scenario 2
    for day in range(transactions_df.TX_TIME_DAYS.max()):
        
        compromised_terminals = terminal_profiles_table.TERMINAL_ID.sample(n=2, random_state=day)
        
        compromised_transactions=transactions_df[(transactions_df.TX_TIME_DAYS>=day) & 
                                                    (transactions_df.TX_TIME_DAYS<day+28) & 
                                                    (transactions_df.TERMINAL_ID.isin(compromised_terminals))]
                            
        transactions_df.loc[compromised_transactions.index,'TX_FRAUD']=1
        transactions_df.loc[compromised_transactions.index,'TX_FRAUD_SCENARIO']=2
    
    nb_frauds_scenario_2=transactions_df.TX_FRAUD.sum()-nb_frauds_scenario_1
    print("Number of frauds from scenario 2: "+str(nb_frauds_scenario_2))
    
    # Scenario 3
    for day in range(transactions_df.TX_TIME_DAYS.max()):
        
        compromised_customers = customer_profiles_table.CUSTOMER_ID.sample(n=3, random_state=day).values
        
        compromised_transactions=transactions_df[(transactions_df.TX_TIME_DAYS>=day) & 
                                                    (transactions_df.TX_TIME_DAYS<day+14) & 
                                                    (transactions_df.CUSTOMER_ID.isin(compromised_customers))]
        
        nb_compromised_transactions=len(compromised_transactions)
        
        
        random.seed(day)
        index_fauds = random.sample(list(compromised_transactions.index.values),k=int(nb_compromised_transactions/3))
        
        transactions_df.loc[index_fauds,'TX_AMOUNT']=transactions_df.loc[index_fauds,'TX_AMOUNT']*5
        transactions_df.loc[index_fauds,'TX_FRAUD']=1
        transactions_df.loc[index_fauds,'TX_FRAUD_SCENARIO']=3
        
                             
    nb_frauds_scenario_3=transactions_df.TX_FRAUD.sum()-nb_frauds_scenario_2-nb_frauds_scenario_1
    print("Number of frauds from scenario 3: "+str(nb_frauds_scenario_3))
    
    return transactions_df

%time transactions_df = add_frauds(customer_profiles_table, terminal_profiles_table, transactions_df)

Number of frauds from scenario 1: 1114
TRANSACTION_ID         int64
TX_DATETIME           object
CUSTOMER_ID            int64
TERMINAL_ID            int64
TX_AMOUNT            float64
TX_TIME_SECONDS        int64
TX_TIME_DAYS           int64
TX_FRAUD               int64
TX_FRAUD_SCENARIO      int64
dtype: object
Number of frauds from scenario 2: 11422
Number of frauds from scenario 3: 5399
CPU times: user 7.72 s, sys: 688 ms, total: 8.41 s
Wall time: 6.52 s


In [58]:
transactions_df.TX_FRAUD.mean()

0.008922885572139303

In [59]:
transactions_df.TX_FRAUD.sum()

17935

In [60]:
transactions_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,0,2018-04-01 23:16:50,0,1996,89.217811,83810,0,0,0
1,1,2018-04-01 04:03:12,0,1370,91.480598,14592,0,0,0
2,2,2018-04-01 00:54:38,0,2694,42.717394,3278,0,0,0
3,3,2018-04-01 10:00:48,0,9071,118.238827,36048,0,0,0
4,4,2018-04-02 08:54:58,0,669,130.582089,32098,1,0,0


In [62]:
transactions_df[transactions_df.TX_FRAUD_SCENARIO==1].shape

(1102, 9)

In [63]:
transactions_df[transactions_df.TX_FRAUD_SCENARIO==2].shape

(11400, 9)

In [64]:
transactions_df[transactions_df.TX_FRAUD_SCENARIO==3].shape

(5433, 9)

In [76]:
%%capture

sns.set(style='darkgrid')
sns.set(font_scale=1.4)

fraud_and_transactions_stats_fig = plt.gcf()

fraud_and_transactions_stats_fig.set_size_inches(15, 8)

sns_plot = sns.lineplot(x="TX_TIME_DAYS", y="value", data=tx_stats, hue="stat_type", hue_order=["nb_tx_per_day","nb_fraud_per_day","nb_fraudcard_per_day"], legend=False)

sns_plot.set_title('Total transactions, and number of fraudulent transactions \n and number of compromised cards per day', fontsize=20)
sns_plot.set(xlabel = "Number of days since beginning of data generation", ylabel="Number")

sns_plot.set_ylim([0,300])

labels_legend = ["# transactions per day (/50)", "# fraudulent txs per day", "# fraudulent cards per day"]

sns_plot.legend(loc='upper left', labels=labels_legend,bbox_to_anchor=(1.05, 1), fontsize=15)

In [14]:
#fraud_and_transactions_stats_fig

# Saving data

In [89]:
DIR_OUTPUT = "./simulated-data-raw/"

if not os.path.exists(DIR_OUTPUT):
    os.makedirs(DIR_OUTPUT)

start_date = datetime.strptime("2018-04-01", "%Y-%m-%d")

for day in range(transactions_df.TX_TIME_DAYS.max()+1):
    
    transactions_day = transactions_df[transactions_df.TX_TIME_DAYS==day].sort_values('TX_TIME_SECONDS')
    
    date = start_date + timedelta(days=day)
    filename_output = date.strftime("%Y-%m-%d")+'.pkl'
    
    # Protocol=4 required for Google Colab
    transactions_day.to_pickle(DIR_OUTPUT+filename_output, protocol=4)